<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/src/CLASS/LSTM6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 新段落

In [ ]:
cd gdrive/My Drive/Colab_Notebooks/code


/content/gdrive/My Drive/Colab_Notebooks/code


In [ ]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [2]:
pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 9.2MB/s 
     |████████████████████████████████| 102kB 6.2MB/s 
     |████████████████████████████████| 7.2MB 12.7MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for boto3: filename=boto3-1.16.47-py2.py3-none-any.whl size=128711 sha256=b1a05e2ece73365c052e7b72ddac01b70f25c031a0b7ff59ad77e415849faa8d
  Stored in directory: /root/.cache/pip/wheels/cc/81/49/643c45ae820b8df3dfd4b8218f4990fc6ca8eac47b281bd675
Successfully built boto3
ERROR: botocore 1.19.47 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
from pytorch_pretrained_bert import BertTokenizer,BertModel
import torch
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
a = "i am a dog"
a_tokens = bert_tokenizer.tokenize(a)
print(a_tokens)
a_seq_ids = bert_tokenizer.convert_tokens_to_ids(a_tokens)
print(a_seq_ids)
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
batch_data = torch.Tensor(a_seq_ids).cuda().long().view((1,-1))
out,_ = bert_model(batch_data)
print(out[0].shape)
print(out[0][0][0])

In [19]:
from pytorch_pretrained_bert import BertTokenizer,BertModel
import torch
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
import numpy as np
import csv
label_List = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "FundRaising",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "CleanUp",
        "Hashtags",
        "ContextualInformation",
        "News",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant",
        "OriginalEvent"]
important_list = ["Low",
        "Medium",
        "High",
        "Critical"
       ]
tweets = []
path = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention/data/ll.csv'
with open(path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        tweet_full = line
        tweets.append({
            'id': tweet_full[0],
            'label':tweet_full[1],
            'important':tweet_full[2],
            'text': tweet_full[3].lower(),
            # 'name': tweet_full['user']['name'].split()[0]
            })
tweet_list = []
text_list = []
imformation_list = []
important_list = []
label_list = []
text_list = []
label_list = []
# text_list = np.load('./data/Bert_text1.npy',allow_pickle=True).tolist()
# label_list = np.load('./data/Bert_label1.npy',allow_pickle=True).tolist()
for i in range(len(tweets)):
        if ( i == 3000):
          break
        # if(i < 3000):
        #   continue
        # if(i== 6000):
        #   break
        #text_list.append(tweets[i]['text'])
        a_tokens = bert_tokenizer.tokenize(tweets[i]['text'])
        print(i)
        a_seq_ids = bert_tokenizer.convert_tokens_to_ids(a_tokens)
        batch_data = torch.Tensor(a_seq_ids).cuda().long().view((1,-1))
        bert_model.eval()
        out,_ = bert_model(batch_data)
        #print(type(out[0][0]))
        tweets_vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        tweets_label = eval(tweets[i]['label'])
        tweets_important = tweets[i]['important']
        p = 0
        for j in label_List:
            for k in tweets_label:
                if k == j:
                    tweets_vector[p] = 1
            p+=1
        for q in important_list:
            for l in tweets_important:
                if l == q:
                    tweets_vector[p] = 1
            p+=1
        label_list.append(tweets_vector)
        text_list.append(out[0][0].cpu())


3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199


RuntimeError: ignored

In [4]:
cd /content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention

/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention


In [7]:
a = np.array(label_list)
b = np.array(text_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [8]:
np.save('./data/Bert_label1.npy',a)
np.save('./data/Bert_text1.npy',b)

In [18]:
c = np.load('./data/Bert_label36.npy',allow_pickle=True)

print(c[3599])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
